In [0]:
import numpy as np
import pyspark as ps
import pyspark.pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

#Configuration & Optimisation de l'engin Spark
conf = SparkConf()
#conf.set('spark.executor.memory', '2g')
#conf.set('spark.driver.memory', '2g')

#Pandas configurations
#pd.options.display.max_rows = 5

spark = (
    SparkSession.builder
        .master("local")
        .appName("TransformationExample")
        .config("spark.sql.ansi.enabled", "false")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .getOrCreate()
)

#Chargement
df = pd.read_csv("/Volumes/alpha_cloud_ai_workspace/default/exp-data/cf_asset_mort_init.csv", sep=";")

#Ajout Interet
df = df.assign(tx_interet = lambda x: x.taux_hyp / x.frequence)
df = df.dropna()

#Ajout des 1200 Cashflow
colones = dict()
for i in range(1200):
    colones[f'cash_flow_{i+1}'] = 0
df = df.reindex(columns=list(df.columns) + list(colones.keys()), fill_value=0.0)

display(df)

In [0]:
#Operations
def compute_cashflows(serie):
    maturity = serie['maturity']
    serie[f'cash_flow_1'] = serie['solde_capital']  # Initial cash flow

    for i in range(1, maturity):
        serie[f'cash_flow_{i+1}'] = serie[f'cash_flow_{i}'] - serie['versement'] + (serie[f'cash_flow_{i}'] * serie['tx_interet'])
    return serie

df = df.apply(compute_cashflows, axis='columns')
display(df)

In [0]:
#25-06-30
def projection_cf_mortgage(data):
 
    # Initialize the DataFrame
    cfs = np.zeros((len(data), 1200))
   
    # Iterate over each row in the data
    for j in range(len(data)):
        if data.iloc[j]['maturity']>0:
            tx_interet = data.iloc[j]['taux_hyp'] / data.iloc[j]['frequence']
            maturity = data.iloc[j]['maturity']
            pay_cap = data.iloc[j]['versement']
            notionnel_cul = np.zeros(maturity + 1)
            notionnel_cul[0] = data.iloc[j]['solde_capital']
           
            for k in range(maturity):
                notionnel_cul[k + 1] = notionnel_cul[k] - pay_cap + notionnel_cul[k]*tx_interet
                cfs[j, k] += pay_cap
 
            cfs[j, maturity - 1] += notionnel_cul[maturity]
 
    # Convert the NumPy array to a DataFrame
    cfs_df = pd.DataFrame(cfs, columns=[f'Cash Flow {i}' for i in range(1, cfs.shape[1] + 1)])
   
    # Concatenate the data with the cash flows
    result = pd.concat([data.reset_index(drop=True), cfs_df], axis=1)
    return result

df.info(verbose=False)
display(df)

In [0]:
result = projection_cf_mortgage(df)

In [0]:
df.info(verbose=False)
display(df)